# Delineate Ridge Areas
This notebook will delineate the ridge areas from the transformed rasters.

Ridge area delineation is accomplished with the following operations:
1. clip the transformed rasters
2. apply binary classification to the clipped rasters
3. find the agreement between the binary rasters
4. denoise the agreement raster


## Flow within each class
Each of the geoprocessing steps outlined above is handled by a separate raster processing `class` which all operate in the same basic way
- each class will take the input raster file path, the output directory, and any other required parameters as inputs
- calling the `.execute()` method on the class instance will perform all required operations for the class including writing the output raster to disk in the output directory


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd

from scrollstats.delineation import RasterClipper, BinaryClassifier, RasterAgreementAssessor, RasterDenoiser

## Prepare Inputs

### Set input paths and output directory

In [ ]:
# Input Paths
bend_path = Path("example_data/input/LBR_025_bend.geojson")
profc_path = Path("example_data/output/LBR_025_dem_profc45px.tif")
rt_path = Path("example_data/output/LBR_025_dem_rt45px.tif")

raster_paths = [profc_path, rt_path]

# Output Directory
output_dir = Path("example_data/output")

### Plot bend boundary

In [ ]:
bend = gpd.GeoDataFrame.from_file(bend_path)
bend.explore()

## Process Rasters

### Clip tranformed rasters
- clip both the profile curvature and residual topography rasters to the bend boundary

In [ ]:
clip_paths = []
for raster_path in raster_paths:

    geom = bend["geometry"][0] # RasterClipper requires a shapely polygon, not geodataframe

    rc = RasterClipper(raster_path, geom, output_dir)
    clip_path = rc.execute()
    
    clip_paths.append(clip_path)

In [ ]:
# Plot clip results
rt = rasterio.open(rt_path).read(1)
rt_clip = rasterio.open(clip_paths[1]).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(rt)
ax1.set_axis_off()
ax1.set_title("Original Raster")


mapper = ax2.imshow(rt_clip)
ax2.set_axis_off()
ax2.set_title("Clipped Raster");


### Create binary classification
- apply binary classification to both of the clipped rasters 

    ```
    if px > 0:
        px == 1
    if px <= 0:
        px == 0
    ```

In [ ]:
binclass_paths = []
for clip_path in clip_paths:
    bc = BinaryClassifier(clip_path, 0, output_dir)
    binclass_path = bc.execute()
    binclass_paths.append(binclass_path)

In [ ]:
# Plot binary classification results
profc = rasterio.open(clip_paths[0]).read(1)
profc_bin = rasterio.open(binclass_paths[0]).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(profc)
ax1.set_axis_off()
ax1.set_title("Original Raster")

mapper = ax2.imshow(profc_bin)
ax2.set_axis_off()
ax2.set_title("Binary Raster");

### Create agreement raster

In [ ]:
# Set parameters
bend_id = "LBR_025"
profc_binclass_path = binclass_paths[0]
rt_binclass_path = binclass_paths[1]

ra = RasterAgreementAssessor(profc_binclass_path, rt_binclass_path, bend_id, output_dir)
agreement_path = ra.execute()

In [ ]:
# Plot agreement results
profc = rasterio.open(profc_binclass_path).read(1)
rt = rasterio.open(rt_binclass_path).read(1)
agr = rasterio.open(agreement_path).read(1)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 6))

mapper = ax1.imshow(profc, alpha=0.7)
ax1.set_axis_off()
ax1.set_title("Profile Curvature")

mapper = ax2.imshow(agr)
ax2.set_axis_off()
ax2.set_title("Agreement")

mapper = ax3.imshow(rt, alpha=0.7)
ax3.set_axis_off()
ax3.set_title("Residual Topography");

### Denoise agreement raster

In [ ]:
# Set thresholds for denoising
small_feats_size = 500
elongation_threshold = 80

dn = RasterDenoiser(agreement_path, small_feats_size, elongation_threshold, output_dir)
denoise_path = dn.execute()

In [ ]:
# Plot denoising results
agr = rasterio.open(agreement_path).read(1)
denoise = rasterio.open(denoise_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(agr)
ax1.set_axis_off()
ax1.set_title("Agreement")

mapper = ax2.imshow(denoise)
ax2.set_axis_off()
ax2.set_title("Denoised");

## Move on to [`CalculateRidgeMetrics.ipynb`](CalculateRidgeMetrics.ipynb)
- All of the required datasets for ridge metric calculation have now been created. 
- Open the notebook `CalculateRidgeMetrics.ipynb` to calculate ridge metrics.